In [2]:
import geopandas as gpd
from shapely.geometry import LineString, Point
import matplotlib.pyplot as plt
import pandas as pd
from shapely import geos
from shapely.wkt import loads

In [3]:
df = pd.read_excel('accidents.xlsx', sheet_name = 'ONSV')
#df.head()

In [4]:
vehicle_columns = ['VEHICULO 1', 'VEHICULO 2', 'VEHICULO 3', 'VEHICULO 4']

def count_non_empty(row):
    return sum(1 for col in vehicle_columns if pd.notna(row[col]))

In [5]:
df['# of vehicles in accident'] = df.apply(count_non_empty, axis=1)
df.head()

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,GRAVEDAD,TOTAL FALLECIDOS,TOTAL LESIONADOS,VEHICULO 1,VEHICULO 2,VEHICULO 3,VEHICULO 4,LATITUD,LONGITUD,# of vehicles in accident
0,1,UPIAT - HUACHO,LIMA,HUAURA,VEGUETA,10/01/2021,2021,January,Sunday,05:20,...,FATAL,1,0,NO IDENTIFICADO,NaN,NaN,NaN,-11.030084,-77.592648,1
1,2,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,10/01/2021,2021,January,Sunday,20:30,...,FATAL,1,1,AUTOMOVIL,AUTOMOVIL,NaN,NaN,-11.605631,-77.241772,2
2,3,UPIAT - HUACHO,LIMA,HUAURA,HUACHO,12/01/2021,2021,January,Tuesday,19:39,...,FATAL,1,0,AUTOMOVIL,AUTOMOVIL,AUTOMOVIL,NaN,-11.184285,-77.576240,3
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,FATAL,1,0,REMOLCADOR - SEMIREMOLQUE,NaN,NaN,NaN,-12.054372,-77.142617,1
4,5,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,24/01/2021,2021,January,Sunday,20:10,...,FATAL,1,0,OMNIBUS,BICICLETA,NaN,NaN,-11.881892,-77.126265,2


In [6]:
# Convert the DataFrame to a GeoDataFrame with LineString geometry
gdf_rta = gpd.GeoDataFrame(df)

In [7]:
gdf_rta['geometry'] = gpd.points_from_xy(gdf_rta['LONGITUD'], gdf_rta['LATITUD'])

In [8]:
gdf_rta = gdf_rta.drop(columns=['LONGITUD', 'LATITUD'])

In [9]:
gdf_rta.head()

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,TIPO DE SINIESTRO,GRAVEDAD,TOTAL FALLECIDOS,TOTAL LESIONADOS,VEHICULO 1,VEHICULO 2,VEHICULO 3,VEHICULO 4,# of vehicles in accident,geometry
0,1,UPIAT - HUACHO,LIMA,HUAURA,VEGUETA,10/01/2021,2021,January,Sunday,05:20,...,ATROPELLO FUGA,FATAL,1,0,NO IDENTIFICADO,NaN,NaN,NaN,1,POINT (-77.59265 -11.03008)
1,2,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,10/01/2021,2021,January,Sunday,20:30,...,CHOQUE,FATAL,1,1,AUTOMOVIL,AUTOMOVIL,NaN,NaN,2,POINT (-77.24177 -11.60563)
2,3,UPIAT - HUACHO,LIMA,HUAURA,HUACHO,12/01/2021,2021,January,Tuesday,19:39,...,CHOQUE,FATAL,1,0,AUTOMOVIL,AUTOMOVIL,AUTOMOVIL,NaN,3,POINT (-77.57624 -11.18429)
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,CAÍDA DE PASAJERO,FATAL,1,0,REMOLCADOR - SEMIREMOLQUE,NaN,NaN,NaN,1,POINT (-77.14262 -12.05437)
4,5,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,24/01/2021,2021,January,Sunday,20:10,...,DESPISTE,FATAL,1,0,OMNIBUS,BICICLETA,NaN,NaN,2,POINT (-77.12627 -11.88189)


In [10]:
gdf_rta.crs = 'EPSG:4326'

In [11]:
df2 = pd.read_excel('failures_detected_output.xlsx')
df2.head()

,analisis_name,id_clase,clases,id_tramo_falla,distances,coordenada_inicio,latitud_inicio,longitud_inicio,coordenada_final,latitude_final,longitud_final,base,area,boxes
0,PE20230403T-F-P06,0,Desenfoque Linea Blanca,36,14.636535,"(-11.044372757440476, -77.5891362827381)",-11.044373,-77.589136,"(-11.044372757440476, -77.5891362827381)",-11.044373,-77.589136,910.592651,13327.920895,1
1,PE20230403T-F-P06,0,Desenfoque Linea Blanca,37,15.642927,"(-11.063438333905678, -77.5882053331044)",-11.063438,-77.588205,"(-11.063438333905678, -77.5882053331044)",-11.063438,-77.588205,792.107620,12390.881812,1
2,PE20230403T-F-P06,0,Desenfoque Linea Blanca,38,0.433942,"(-11.071720774496338, -77.58487642353481)",-11.071721,-77.584876,"(-11.071720774496338, -77.58487642353481)",-11.071721,-77.584876,590.992355,256.456536,1
3,PE20230403T-F-P06,0,Desenfoque Linea Blanca,39,0.430801,"(-11.073483825663919, -77.58417556215659)",-11.073484,-77.584176,"(-11.073483825663919, -77.58417556215659)",-11.073484,-77.584176,909.705734,391.902126,1
4,PE20230403T-F-P06,0,Desenfoque Linea Blanca,40,0.407377,"(-11.087819480998169, -77.58324718635531)",-11.087819,-77.583247,"(-11.087819480998169, -77.58324718635531)",-11.087819,-77.583247,765.128403,311.695672,1


In [12]:
# Convert the DataFrame to a GeoDataFrame with LineString geometry
gdf_defects = gpd.GeoDataFrame(df2)

In [13]:
gdf_defects['geometry'] = gpd.points_from_xy(gdf_defects['longitud_inicio'], gdf_defects['latitud_inicio'])

In [14]:
gdf_defects.columns

Index(['analisis_name', 'id_clase', 'clases', 'id_tramo_falla', 'distances',
       'coordenada_inicio', 'latitud_inicio', 'longitud_inicio',
       'coordenada_final', 'latitude_final', 'longitud_final', 'base', 'area',
       'boxes', 'geometry'],
      dtype='object')

In [15]:
gdf_defects = gdf_defects.drop(columns=['coordenada_inicio','coordenada_final', 'latitude_final', 'longitud_final'])

In [16]:
gdf_defects.head()

,analisis_name,id_clase,clases,id_tramo_falla,distances,latitud_inicio,longitud_inicio,base,area,boxes,geometry
0,PE20230403T-F-P06,0,Desenfoque Linea Blanca,36,14.636535,-11.044373,-77.589136,910.592651,13327.920895,1,POINT (-77.58914 -11.04437)
1,PE20230403T-F-P06,0,Desenfoque Linea Blanca,37,15.642927,-11.063438,-77.588205,792.107620,12390.881812,1,POINT (-77.58821 -11.06344)
2,PE20230403T-F-P06,0,Desenfoque Linea Blanca,38,0.433942,-11.071721,-77.584876,590.992355,256.456536,1,POINT (-77.58488 -11.07172)
3,PE20230403T-F-P06,0,Desenfoque Linea Blanca,39,0.430801,-11.073484,-77.584176,909.705734,391.902126,1,POINT (-77.58418 -11.07348)
4,PE20230403T-F-P06,0,Desenfoque Linea Blanca,40,0.407377,-11.087819,-77.583247,765.128403,311.695672,1,POINT (-77.58325 -11.08782)


In [17]:
gdf_defects.crs = 'EPSG:4326'

In [18]:
gdf_rta = gdf_rta.to_crs('EPSG:3857')
gdf_defects = gdf_defects.to_crs('EPSG:3857')


In [19]:
print(gdf_rta.crs)
print(gdf_defects.crs)

EPSG:3857
EPSG:3857


In [20]:
#buffer around the accidents
gdf_rta['geometry'] = gdf_rta['geometry'].buffer(10)
gdf_rta.head()

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,TIPO DE SINIESTRO,GRAVEDAD,TOTAL FALLECIDOS,TOTAL LESIONADOS,VEHICULO 1,VEHICULO 2,VEHICULO 3,VEHICULO 4,# of vehicles in accident,geometry
0,1,UPIAT - HUACHO,LIMA,HUAURA,VEGUETA,10/01/2021,2021,January,Sunday,05:20,...,ATROPELLO FUGA,FATAL,1,0,NO IDENTIFICADO,NaN,NaN,NaN,1,"POLYGON ((-8637564.096 -1235518.571, -8637564...."
1,2,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,10/01/2021,2021,January,Sunday,20:30,...,CHOQUE,FATAL,1,1,AUTOMOVIL,AUTOMOVIL,NaN,NaN,2,"POLYGON ((-8598504.764 -1300859.092, -8598504...."
2,3,UPIAT - HUACHO,LIMA,HUAURA,HUACHO,12/01/2021,2021,January,Tuesday,19:39,...,CHOQUE,FATAL,1,0,AUTOMOVIL,AUTOMOVIL,AUTOMOVIL,NaN,3,"POLYGON ((-8635737.574 -1253011.901, -8635737...."
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,CAÍDA DE PASAJERO,FATAL,1,0,REMOLCADOR - SEMIREMOLQUE,NaN,NaN,NaN,1,"POLYGON ((-8587466.815 -1351896.932, -8587466...."
4,5,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,24/01/2021,2021,January,Sunday,20:10,...,DESPISTE,FATAL,1,0,OMNIBUS,BICICLETA,NaN,NaN,2,"POLYGON ((-8585646.577 -1332269.933, -8585646...."


In [23]:
gdf_rta.shape[0]

121

In [28]:
#blackspot_analysis = gpd.sjoin(gdf_defects, gdf_rta, predicate='within', lsuffix= '_defects', rsuffix = '_fatalities')
blackspot_analysis = gpd.sjoin(gdf_rta, gdf_defects, predicate='contains')

In [29]:
blackspot_analysis.head(100)

,N°,UPIAT *,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA,AÑO,MES,DIA,HORA,...,analisis_name,id_clase,clases,id_tramo_falla,distances,latitud_inicio,longitud_inicio,base,area,boxes
3,4,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,17/01/2021,2021,January,Sunday,08:00,...,PE20230416M-F-P06,0,Desenfoque Linea Blanca,13,0.173064,-12.054305,-77.142675,387.975769,67.144734,1
9,10,UPIAT - HUACHO,LIMA,HUAURA,SANTA MARIA,04/03/2021,2021,March,Thursday,06:28,...,PE20230403M-F-P11,0,Desenfoque Linea Blanca,5,17.207931,-11.112783,-77.592410,672.329636,11569.402301,1
19,20,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,25/04/2021,2021,April,Sunday,02:15,...,PE20230416M-F-P05,0,Piel de Cocodrilo,3,45.278309,-12.050022,-77.139718,907.700394,46203.012815,8
21,22,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,29/04/2021,2021,April,Thursday,16:00,...,PE20230416M-F-P03,0,Desenfoque Linea Blanca,103,11.170553,-11.994462,-77.126014,247.783813,2767.882215,1
25,26,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,17/05/2021,2021,May,Monday,16:10,...,PE20230416M-F-P02,0,"Protuberancia, Bache",99,12.214683,-11.904137,-77.128520,207.583008,2535.560716,1
27,28,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,05/06/2021,2021,June,Saturday,19:30,...,PE20230416M-F-P03,0,Intervalo Lineal Transversal,80,0.340532,-11.964844,-77.126666,148.770447,50.661024,1
34,35,UPIAT - CALLAO,CALLAO,CALLAO,VENTANILLA,25/06/2021,2021,June,Friday,05:35,...,PE20230416M-F-P09,0,Piel de Cocodrilo,25,19.019896,-11.873081,-77.127180,915.230236,16241.884542,3
53,54,UPIAT - HUACHO,LIMA,HUARAL,CHANCAY,07/11/2021,2021,November,Sunday,04:30,...,PE20230403M-F-P04,0,Piel de Cocodrilo,39,13.466999,-11.595672,-77.254922,867.635765,12036.385361,2
59,60,UPIAT - HUACHO,LIMA,BARRANCA,SUPE,27/11/2021,2021,November,Saturday,01:10,...,PE20230403M-F-P14,0,Desenfoque Linea Blanca,37,15.957515,-10.802256,-77.703451,467.552376,7460.974056,1
75,76,UPIAT - CALLAO,CALLAO,CALLAO,CALLAO,22/01/2022,2022,January,Saturday,06:00,...,PE20230416M-F-P06,0,Grieta Lineal Transversal,125,7.483147,-12.046644,-77.132144,203.663177,1524.041538,1


In [100]:
blackspot_analysis.to_csv("blackspot_analysis.csv")